In [ ]:
import sys
import os
sys.path.insert(0, '/Users/lerinalaluf/Desktop/SIP_2021/kaepora/src')
os.chdir('/Users/lerinalaluf/Desktop/SIP_2021/kaepora/src')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import kaepora as kpora
import numpy as np

In [ ]:
spectra_querry = ["SELECT * from Spectra inner join Events ON Spectra.SN = Events.SN"]

In [ ]:
spec_array = kpora.grab(spectra_querry[0], verbose=True, db_file = '../data/kaepora_v1_DEV.db')

In [ ]:
MIN=1566
MAX=2605

num_cols = len(spec_array)
num_rows = MAX-MIN

grid = [[None]*(num_rows) for z in range(num_cols)]

for x in range(num_cols):
    z=0
    for y in range(MIN, MAX):
        grid[x][z] = spec_array[x].flux[y]
        z+=1

np.savetxt('Flux Data.txt', grid)    